In [ ]:
def special_tag(df):
    total = []
    small = []
    numlist = df['number'].tolist()
    taglist = df['specific_tag'].tolist()
    titlelist = df['title'].tolist()
    artistlist = df['artist'].tolist()
    labellist = df['label'].tolist()

    for i in range(len(df)-1):
        if titlelist[i] == titlelist[i+1] and \
           artistlist[i] == artistlist[i+1] and \
          labellist[i] == labellist[i+1]:
            for k in taglist[i]:
                if k in small:
                    continue
                else:
                    small.append(k)
        else:
            for k in taglist[i]:
                if k in small:
                    continue
                else:
                    small.append(k)
            total.append(small)
            small = []

    num1 = numlist[-1]
    #마지막 행 처리
    for j in range(num1):
        for k in taglist[-j-1]:
            if k in small:
                continue
            else:
                small.append(k)

    total.append(small)
    return total

In [ ]:
def preprocessing(df):
    # 문자열에서 리스트로 변환
    for i in range(len(df)):
        df['specific_tag'][i] = eval(df['specific_tag'][i])

    #number가 1과 1이 아닌 것 구분
    df_1 = df[df['number']==1]
    df_1.drop('tag', axis=1, inplace=True)
    df_2 = df[df['number']!=1]
    df_2.drop('tag', axis=1, inplace=True)

    #number와 title, label로 정렬, 중복 제거된 데이터와 순서가 같아야 함.
    df_2 = df_2.sort_values(by=['number', 'title'])
    unique_pl = df_2[['title','artist', 'label']].value_counts().to_frame(name='number')
    unique_pl = unique_pl.sort_values(by=['number', 'title'])

    #special tag가 합쳐진 리스트 뽑기
    special_taglist = special_tag(df_2)
 
    #likes 수 중복되지 않게 리스트 만들기
    numlist = df_2['number'].tolist()
    rem_dup_list = remove_duplicate(numlist)
    likelist = df_2['likes'].tolist()
    rem_dup_likelist = []
    plus = 0
    for i in rem_dup_list:
        rem_dup_likelist.append(likelist[plus])
        plus += i

    # unique_pl에 special_tag, likes 붙이기
    unique_pl.reset_index(inplace=True)
    unique_pl['likes'] = rem_dup_likelist
    unique_pl['specific_tag'] = special_taglist
    # 변수 순서 바꿔주기
    unique_pl = unique_pl[['title', 'artist', 'specific_tag', 'label', 'number', 'likes']]

    df_3 = pd.concat([df_1, unique_pl], ignore_index = True)
    return df_3

In [ ]:
def remove_duplicate(lst):
    no_duplicate = []
    for i in lst:
        plus = 1
        no_duplicate.append(i)
        while plus < i:
            lst.remove(lst[i+plus-1])
            plus += 1
    return no_duplicate

In [ ]:
df = preprocessing(df) #number까지 만든 파일 넣기

In [ ]:
df = df.sort_values(['title','artist','label'])

copy = df
titlelst = df['title'].tolist()
artistlst = df['artist'].tolist()
labellst = df['label'].tolist()
copy = copy['number'].tolist()
remove_num = []

for i in range(len(df)-1):
    if titlelst[i] == titlelst[i+1] \
    and artistlst[i] == artistlst[i+1] \
    and labellst[i] == labellst[i+1]:
        copy[i] += copy[i+1]
        remove_num.append(i+1)

final = df
final = final.reset_index()
final['number'] = copy
final.drop(remove_num, axis=0, inplace=True)
final.drop(['index', 'search'], axis=1, inplace=True)